In [2]:
import itertools
import random
import ast
import pandas as pd
import numpy as np
from collections import defaultdict

In [8]:
# Load in all metric patterns from combos 5, 6, and 7
metric_5 = pd.read_csv('stim_conditions/combo_chart_5.csv')
metric_6 = pd.read_csv('stim_conditions/combo_chart_6.csv')
metric_7 = pd.read_csv('stim_conditions/combo_chart_7.csv')
metric_patterns = pd.concat([metric_5, metric_6, metric_7], axis=0)
simple_patterns = [[int(x) for x in ast.literal_eval(pattern)] for pattern in metric_patterns['simple'].dropna().tolist()]
complex_patterns = [[int(x) for x in ast.literal_eval(pattern)] for pattern in metric_patterns['complex'].dropna().tolist()]
simple_silence_patterns = [[int(x) for x in ast.literal_eval(pattern)] for pattern in metric_patterns['simple_silence'].dropna().tolist()]
complex_silence_patterns = [[int(x) for x in ast.literal_eval(pattern)] for pattern in metric_patterns['complex_silence'].dropna().tolist()]

In [9]:
# Remove nan values from simple patterns
simple_patterns = [pattern for pattern in simple_patterns if pattern != [np.nan]]
complex_patterns = [pattern for pattern in complex_patterns if pattern != [np.nan]]
simple_silence_patterns = [pattern for pattern in simple_silence_patterns if pattern != [np.nan]]
complex_silence_patterns = [pattern for pattern in complex_silence_patterns if pattern != [np.nan]]


In [10]:
metric_patterns

,simple,simple_silence,complex,complex_silence
0,"[1, 1, 2, 4, 4]","[0, 0, 1, 3, 3]","[1, 1, 3, 3, 4]","[0, 0, 2, 2, 3]"
1,"[1, 2, 1, 4, 4]","[0, 1, 0, 3, 3]","[1, 1, 3, 4, 3]","[0, 0, 2, 3, 2]"
2,"[1, 3, 1, 3, 4]","[0, 2, 0, 2, 3]","[1, 1, 4, 2, 4]","[0, 0, 3, 1, 3]"
3,"[1, 3, 2, 2, 4]","[0, 2, 1, 1, 3]","[1, 1, 4, 3, 3]","[0, 0, 3, 2, 2]"
4,"[1, 3, 3, 1, 4]","[0, 2, 2, 0, 3]","[1, 1, 4, 4, 2]","[0, 0, 3, 3, 1]"
...,...,...,...,...
282,"[1, 2, 1, 3, 1, 1, 3]","[0, 1, 0, 2, 0, 0, 2]","[4, 1, 1, 1, 2, 2, 1]","[3, 0, 0, 0, 1, 1, 0]"
283,"[1, 2, 1, 3, 1, 2, 2]","[0, 1, 0, 2, 0, 1, 1]","[4, 1, 1, 1, 3, 1, 1]","[3, 0, 0, 0, 2, 0, 0]"
284,"[1, 2, 1, 3, 1, 3, 1]","[0, 1, 0, 2, 0, 2, 0]","[4, 1, 1, 3, 1, 1, 1]","[3, 0, 0, 2, 0, 0, 0]"
285,"[1, 2, 1, 4, 1, 1, 2]","[0, 1, 0, 3, 0, 0, 1]","[4, 1, 2, 2, 1, 1, 1]","[3, 0, 1, 1, 0, 0, 0]"


In [11]:
# Turn patterns into dataframes
simple_patterns_df = pd.DataFrame(simple_patterns)
# Get counts of 1, 2, 3, 4 in each of the 6 columns
simple_counts = simple_patterns_df.apply(pd.Series.value_counts, axis=0).fillna(0)
simple_counts = simple_counts.reindex(range(1, 5)).fillna(0)
simple_counts = simple_counts.astype(int)

# Turn patterns into dataframes
complex_patterns_df = pd.DataFrame(complex_patterns)
# Get counts of 1, 2, 3, 4 in each of the 6 columns
complex_counts = complex_patterns_df.apply(pd.Series.value_counts, axis=0).fillna(0)
complex_counts = complex_counts.reindex(range(1, 5)).fillna(0)
complex_counts = complex_counts.astype(int)

In [1409]:
# Define the values you want to filter by
filter_value_1 = 4
filter_value_2 = 1
filter_value_3 = 1
filter_value_4 = 3

# Apply the filters using the defined variables
simple_patterns_all = simple_patterns_df[(simple_patterns_df == filter_value_1).sum(axis=1) > 0]
simple_patterns_all = simple_patterns_all[(simple_patterns_all == filter_value_2).sum(axis=1) > 0]
simple_patterns_all = simple_patterns_all[~((simple_patterns_all[1] == filter_value_3) & (simple_patterns_all[3] == filter_value_4))]
simple_patterns_all = simple_patterns_all.reset_index(drop=True)

complex_patterns_all = complex_patterns_df[(complex_patterns_df == filter_value_1).sum(axis=1) > 0]
complex_patterns_all = complex_patterns_all[(complex_patterns_all == filter_value_2).sum(axis=1) > 0]
complex_patterns_all = complex_patterns_all[~((complex_patterns_all[1] == filter_value_3) & (complex_patterns_all[3] == filter_value_4))]

# make patterns into a list of lists
simple_patterns_all = simple_patterns_all.values.tolist()
complex_patterns_all = complex_patterns_all.values.tolist()


In [25]:
complex_patterns_all = complex_patterns_df.values.tolist()
simple_patterns_all = simple_patterns_df.values.tolist()

In [26]:
# Function to generate all unique combinations of four words without repetitions
def generate_unique_combinations(words):
    return list(itertools.permutations(words, 4))

def arrange_syllables(unique_combinations, previous_combination, used_combinations, cycle_limit):
    random.shuffle(unique_combinations)  # Shuffle to ensure a random selection each time
    for combo in unique_combinations:
        # Check if we have used all combinations in this cycle
        if len(used_combinations) >= cycle_limit:
            used_combinations.clear()  # Reset for a new cycle

        # Ensure new combo is not the same as previous and does not start with previous combo's last element
        if combo != previous_combination \
           and (not previous_combination or combo[0] != previous_combination[-1]) \
           and combo not in used_combinations:
            return combo
    return None  # This should ideally not occur

def interweave(syllables, silence_pattern):
    # Ensure syllables are broken into individual letters
    flattened_syllables = [letter for word in syllables for letter in word]
    
    # Create the interweaved sequence
    interweaved = []
    for letter, silence in zip(flattened_syllables, silence_pattern):
        interweaved.append(letter)
        interweaved.append(silence)
    
    return interweaved

def pick_pattern_template_and_syllables(condition):

    # Choose appropriate patterns based on the condition
    patterns = simple_patterns_all if condition == 'simple' else complex_patterns_all
    silence_patterns = simple_silence_patterns if condition == 'simple' else complex_silence_patterns

    # Flatten the patterns if they are lists of lists
    patterns = [item for sublist in patterns for item in sublist]
    silence_patterns = [item for sublist in silence_patterns for item in sublist]

    # Randomly select 96 patterns and 96 silence patterns
    selected_patterns = random.choices(patterns, k=576)
    selected_silence_patterns = random.choices(silence_patterns, k=576)

    # Combine selected patterns into one string each
    pattern = ''.join([str(x) for x in selected_patterns])

    return pattern

In [27]:
words = ['ABC', 'DEF', 'GHI', 'JKL']
unique_combinations = generate_unique_combinations(words)
arranged_syllables = []
previous_combination = None
used_combinations = set()

cycle_limit = len(unique_combinations)

In [32]:
for _ in range(96):  # We aim for 48 sets
    new_set = arrange_syllables(unique_combinations, previous_combination, used_combinations, cycle_limit)
    if new_set:
        arranged_syllables.append(new_set)
        used_combinations.add(new_set)
        previous_combination = new_set
        # flatten the list of lists
flattened_syllables = [letter for word in arranged_syllables for letter in word]
print('flattened_syllables:', flattened_syllables)

flattened_syllables: ['JKL', 'DEF', 'ABC', 'GHI', 'DEF', 'ABC', 'JKL', 'GHI', 'DEF', 'GHI', 'JKL', 'ABC', 'GHI', 'JKL', 'ABC', 'DEF', 'ABC', 'DEF', 'JKL', 'GHI', 'JKL', 'GHI', 'ABC', 'DEF', 'JKL', 'ABC', 'GHI', 'DEF', 'ABC', 'JKL', 'GHI', 'DEF', 'ABC', 'DEF', 'GHI', 'JKL', 'ABC', 'JKL', 'DEF', 'GHI', 'JKL', 'DEF', 'GHI', 'ABC', 'GHI', 'ABC', 'JKL', 'DEF', 'GHI', 'JKL', 'DEF', 'ABC', 'DEF', 'JKL', 'ABC', 'GHI', 'DEF', 'ABC', 'GHI', 'JKL', 'GHI', 'DEF', 'ABC', 'JKL', 'DEF', 'JKL', 'GHI', 'ABC', 'GHI', 'ABC', 'DEF', 'JKL', 'DEF', 'GHI', 'ABC', 'JKL', 'GHI', 'DEF', 'JKL', 'ABC', 'JKL', 'GHI', 'DEF', 'ABC', 'JKL', 'ABC', 'DEF', 'GHI', 'ABC', 'GHI', 'DEF', 'JKL', 'ABC', 'GHI', 'JKL', 'DEF', 'GHI', 'JKL', 'ABC', 'DEF', 'ABC', 'JKL', 'DEF', 'GHI', 'ABC', 'GHI', 'DEF', 'JKL', 'DEF', 'JKL', 'ABC', 'GHI', 'ABC', 'DEF', 'GHI', 'JKL', 'DEF', 'JKL', 'GHI', 'ABC', 'JKL', 'ABC', 'GHI', 'DEF', 'GHI', 'ABC', 'JKL', 'DEF', 'JKL', 'ABC', 'DEF', 'GHI', 'DEF', 'GHI', 'JKL', 'ABC', 'JKL', 'DEF', 'ABC', 'GHI'

In [33]:
# simple_patterns_all = pd.DataFrame(simple_patterns_all)
# complex_patterns_all = pd.DataFrame(complex_patterns_all)
# # Make a freeuqency table that calculates the postiional frequency of each number in the metric values (1, 2, 3, 4) occurs in which column (1, 2, 3, 4, 5, 6) how many times
# simple_freq_table = defaultdict(lambda: defaultdict(int))
# for i, row in simple_patterns_all.iterrows():
#     for j, value in enumerate(row):
#         simple_freq_table[value][j] += 1

# complex_freq_table = defaultdict(lambda: defaultdict(int))
# for i, row in complex_patterns_all.iterrows():
#     for j, value in enumerate(row):
#         complex_freq_table[value][j] += 1

# display(simple_freq_table)
# display(complex_freq_table)

In [34]:
interweaved_sequence = interweave(flattened_syllables, pick_pattern_template_and_syllables('simple'))
valid_pairs = [
    ('A', 'B'), ('B', 'C'), ('C', 'D'), ('C', 'G'), ('C', 'J'),
    ('D', 'E'), ('E', 'F'), ('F', 'G'), ('F', 'A'), ('F', 'J'),
    ('G', 'H'), ('H', 'I'), ('I', 'J'), ('I', 'A'), ('I', 'D'),
    ('J', 'K'), ('K', 'L'), ('L', 'A'), ('L', 'D'), ('L', 'G')
]
valid_pair_set = set(valid_pairs)
pair_transition_freq = defaultdict(lambda: defaultdict(int))

# Regular counting for all but the last pair
for i in range(0, len(interweaved_sequence) - 2, 2):  # Step through individual pairs
    current_pair = (interweaved_sequence[i], interweaved_sequence[i + 2])
    # Only count if current pair is valid
    if current_pair in valid_pair_set:
        silence_value = interweaved_sequence[i + 1]
        pair_transition_freq[current_pair][silence_value] += 1

# This checks if such a pair (last-first) is a valid pair to be counted
if len(interweaved_sequence) > 2:  # Ensure there are enough elements for a wrap-around
    last_pair = (interweaved_sequence[-2], interweaved_sequence[0])
    last_silence = interweaved_sequence[-1]
    if last_pair in valid_pair_set:
        pair_transition_freq[last_pair][last_silence] += 1

# Convert the frequency table to a DataFrame for analysis and display
df_pair_transitions = pd.DataFrame(pair_transition_freq).T.fillna(0).astype(int)
df_pair_transitions['Total'] = df_pair_transitions.sum(axis=1)
df_pair_transitions.sort_values('Total', ascending=False, inplace=True)
df_pair_transitions = df_pair_transitions.sort_index(axis=1)  # Sorting for better visualization and consistency

# Display the refined frequency table
print("Simple Transitional Frequencies")
display(df_pair_transitions)


Simple Transitional Frequencies


,,.,0,1,2,3,4,Total,a,n
J,K,0,0,34,17,7,8,71,0,5
A,B,0,0,27,17,12,4,71,0,11
K,L,66,0,0,0,0,0,71,5,0
G,H,0,0,28,20,10,5,71,0,8
B,C,60,0,0,0,0,0,71,11,0
H,I,63,0,0,0,0,0,71,8,0
E,F,63,0,0,0,0,0,71,8,0
D,E,0,0,22,24,11,6,71,0,8
C,G,0,19,0,0,0,0,26,0,7
F,A,0,24,0,0,0,0,25,0,1


In [36]:
interweaved_sequence = interweave(flattened_syllables, pick_pattern_template_and_syllables('complex'))
print(pick_pattern_template_and_syllables('complex'))
valid_pairs = [
    ('A', 'B'), ('B', 'C'), ('C', 'D'), ('C', 'G'), ('C', 'J'),
    ('D', 'E'), ('E', 'F'), ('F', 'G'), ('F', 'A'), ('F', 'J'),
    ('G', 'H'), ('H', 'I'), ('I', 'J'), ('I', 'A'), ('I', 'D'),
    ('J', 'K'), ('K', 'L'), ('L', 'A'), ('L', 'D'), ('L', 'G')
]
valid_pair_set = set(valid_pairs)
pair_transition_freq = defaultdict(lambda: defaultdict(int))

# Regular counting for all but the last pair
for i in range(0, len(interweaved_sequence) - 2, 2):  # Step through individual pairs
    current_pair = (interweaved_sequence[i], interweaved_sequence[i + 2])
    # Only count if current pair is valid
    if current_pair in valid_pair_set:
        silence_value = interweaved_sequence[i + 1]
        pair_transition_freq[current_pair][silence_value] += 1

# This checks if such a pair (last-first) is a valid pair to be counted
if len(interweaved_sequence) > 2:  # Ensure there are enough elements for a wrap-around
    last_pair = (interweaved_sequence[-2], interweaved_sequence[0])
    last_silence = interweaved_sequence[-1]
    if last_pair in valid_pair_set:
        pair_transition_freq[last_pair][last_silence] += 1

# Convert the frequency table to a DataFrame for analysis and display
df_pair_transitions = pd.DataFrame(pair_transition_freq).T.fillna(0).astype(int)
df_pair_transitions['Total'] = df_pair_transitions.sum(axis=1)
df_pair_transitions.sort_values('Total', ascending=False, inplace=True)
df_pair_transitions = df_pair_transitions.sort_index(axis=1)  # Sorting for better visualization and consistency

# Display the refined frequency table
print("Complex Transitional Frequencies")

display(df_pair_transitions)


1.0nan2.02.03.01.01.04.04.0nan1.02.01.04.03.01.02.01.0nan2.04.02.02.01.03.03.01.01.01.02.03.01.0nan2.03.04.01.0nan1.03.01.01.04.03.03.01.0nan1.01.03.01.02.01.01.02.01.01.01.01.01.01.03.03.02.01.03.0nan3.02.01.04.02.01.0nan1.01.01.01.01.04.01.02.02.02.01.01.03.04.01.02.02.01.01.01.0nannannannan1.02.02.04.01.02.01.01.01.01.02.02.0nan1.02.03.04.01.01.02.02.02.0nan2.03.01.03.01.01.02.04.01.0nan2.02.02.02.03.02.01.01.02.02.01.03.04.03.04.01.01.01.01.03.02.03.01.0nan2.04.03.01.03.04.01.03.01.0nan2.01.01.01.02.01.01.03.01.04.04.02.02.03.02.02.03.01.01.01.02.02.02.03.02.01.01.01.02.0nan2.01.02.01.02.02.0nannan1.01.02.02.04.01.01.04.02.03.0nan1.01.02.0nan1.02.0nan2.02.01.02.02.0nan4.02.02.03.01.02.02.04.01.01.01.04.0nan2.01.02.0nan3.03.01.01.01.01.01.01.03.01.01.0nan2.02.02.01.0nan1.04.02.02.01.04.02.02.02.0nan1.01.03.01.04.0nan3.0nan1.02.01.03.01.01.0nan1.0nan3.02.03.0nan2.01.04.02.01.01.02.0nan4.02.01.01.01.02.01.0nan1.03.0nan1.01.01.03.0nan1.02.03.01.02.01.01.01.03.03.03.02.0nannannannan2.01

,,.,0,1,2,3,4,Total,a,n
J,K,0,0,20,20,16,7,71,0,8
A,B,0,0,19,20,16,8,71,0,8
K,L,63,0,0,0,0,0,71,8,0
G,H,0,0,26,22,10,3,71,0,10
B,C,63,0,0,0,0,0,71,8,0
H,I,61,0,0,0,0,0,71,10,0
E,F,63,0,0,0,0,0,71,8,0
D,E,0,0,33,17,7,6,71,0,8
C,G,0,25,0,0,0,0,26,0,1
F,A,0,22,0,0,0,0,25,0,3


In [1417]:
# Turn patterns into dataframes
sp_count = pd.DataFrame(simple_patterns_all)
# Get counts of 1, 2, 3, 4 in each of the 6 columns
sp_count = sp_count.apply(pd.Series.value_counts, axis=0).fillna(0)
sp_count = sp_count.reindex(range(1, 5)).fillna(0)
sp_count = sp_count.astype(int)

# Turn patterns into dataframes
cxp_count = pd.DataFrame(complex_patterns_all)
# Get counts of 1, 2, 3, 4 in each of the 6 columns
cxp_count = cxp_count.apply(pd.Series.value_counts, axis=0).fillna(0)
cxp_count = cxp_count.reindex(range(1, 5)).fillna(0)
cxp_count = cxp_count.astype(int)
display(sp_count)
display(cxp_count)

,0,1,2,3,4,5
1,54,81,87,90,81,57
2,33,54,42,45,54,36
3,18,27,9,3,27,18
4,60,3,27,27,3,54


,0,1,2,3,4,5
1,66,51,55,60,57,65
2,36,32,33,35,29,35
3,14,11,17,7,11,14
4,12,34,23,26,31,14


In [ ]:
# Take in the other metric sequences
